In [ ]:
%cd ..
import os
import time

In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

In [ ]:
NUSCENES_PROCESSED_DATA_FULL = "NUSCENES_PROCESSED_DATA_FULL"
if NUSCENES_PROCESSED_DATA_FULL in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA_FULL]
else:
    DATA_DIR = "/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Trainval"
NUSCENES_RAW_DATA_FULL = "NUSCENES_RAW_DATA_FULL"
if NUSCENES_RAW_DATA_FULL in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA_FULL]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval"

In [ ]:
import cv2
def construct_video(frames, size=(1600, 900), base=False, vid_name=None):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig14_"+vid_name if vid_name else f"./outputs/{'fig14_base' if base else 'fig14_optimized'}.avi"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [ ]:
name = 'ScenicWorld' # world name
world = empty_world(name=name)

obj1 = objects[0]
cam = camera
world = world.filter(
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck') | F.like(obj1.type, 'bus')) &
    (F.distance(cam.ego, obj1.trans@cam.timestamp) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -180, -90) &
    F.contained(cam.ego, F.road_segment('road')) &
    F.contained(obj1.trans@cam.time, F.road_segment('road')) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, F.road_direction(obj1.traj@cam.time, obj1.trans@cam.time)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 10)
)

In [ ]:
preprocess(world, DATA_DIR, ['scene-0769-CAM_FRONT'], base=True)

In [ ]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(1)
print(f'optimized query time: {time.time()-optimized_query_start}')

In [ ]:
[x[3] for x in id_time_camId_filename]

In [ ]:
optimized_results = [e[3] for e in id_time_camId_filename]
construct_video(optimized_results)

In [ ]:
preprocess(world, DATA_DIR, ['scene-0769-CAM_FRONT'])


In [ ]:
base_query_start = time.time()
id_time_camId_filename_base = world.get_id_time_camId_filename(1)
print(f'base query time: {time.time()-base_query_start}')

In [ ]:
id_time_camId_filename_base

In [ ]:
base_results = [e[3] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [ ]:
base_results

In [ ]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

In [ ]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")

In [ ]:
investigation = [[217, 1205.0, 198.0, 391.0, 525.0], [104, 1198.0, 192.0, 390.0, 520.0]]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open('/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval/sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490862404.jpg')

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

# Create a Rectangle patch
for obj in investigation:
    obj_id = obj[0]
    x,y,w,h = obj[1:5]
    plt.text(x, y+15, 'obj'+str(obj_id))
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()